# scrapping the original df from the web page and clean it

In [1]:
# imports
import pandas as pd
import lxml

# the wilipedia web page
page = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = page[0]

# filtering the dateframe from the Not-assigned-postal-codes rows
is_assigned = df.Borough != 'Not assigned'
df_filtered = df[is_assigned]

# rearranging the indices
df_filtered.reset_index(inplace=True)

# dropping the excessive column (the former index column)
df_filtered.drop(['index'], axis=1, inplace=True)

# showing the clean dataframe
df_filtered
# NOTE: There will be a warning but it doesn't affect the final result

i:\programs\python\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


# get the coordinates data

In [2]:
# the geocoder coordinates data csv file
add = pd.read_csv('https://cocl.us/Geospatial_data')
add

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


# get the coordinates from the add df to df_filtered

In [3]:
# lists for values to store the lats and lngs
lats = []
lngs = []

# maching the postal codes So that we can append the lats and lngs
for i in range(103):
    for j in range(103):
        if (add.values[i][0] == df_filtered.values[j][0]):   # postal codes are the same
            lats.append(add.values[i][1])
            lngs.append(add.values[i][2])

In [4]:
# append the lists that have the coordinates to the df as columns
df_filtered['Latitude'] = lats
df_filtered['Longitude'] = lngs

# show the final df
df_filtered
# NOTE: There is a warning but it doesn't affect the final result

i:\programs\python\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
i:\programs\python\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.770992,-79.216917
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.773136,-79.239476
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.706876,-79.518188
99,M4Y,Downtown Toronto,Church and Wellesley,43.696319,-79.532242
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.688905,-79.554724
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.739416,-79.588437


# Create a plain map of Toronto

In [8]:
!pip install folium
import folium

# Toronto coordinates
Toronto_lat = 43.651070
Toronto_lng = -79.347015

# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[Toronto_lat, Toronto_lng], zoom_start=12)

You should consider upgrading via the 'i:\programs\python\python.exe -m pip install --upgrade pip' command.


In [9]:
map_Toronto

# Add markers to the map

In [13]:
# add markers to the map
for lat, lng, borough, neighborhood in zip(df_filtered['Latitude'], df_filtered['Longitude'], df_filtered['Borough'], df_filtered['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

# Defining my foursquare credentials

In [14]:
CLIENT_ID = 'V44NEUE3U3W3IDLSRAOBB4MSGI2VJ11JWOQJQ4SCR4CAE4LK'
CLIENT_SECRET = 'PA2V2PQSYUAVTVUSXW0U1SK3PEWOGOHYIH44TXB2GDYLUP0H'
VERSION = '20180605'

# Exploring and Clustering our neighborhoods

# 1. exploring and getting data for venues

In [23]:
# variables for the following dunctions
LIMIT = 103

# A function that repeat the exploring process to all the neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
# import requests to make the GET requests
import requests

# using the function to make a new dataframe
Toronto_venues = getNearbyVenues(names=df_filtered['Neighbourhood'],
                                   latitudes=df_filtered['Latitude'],
                                   longitudes=df_filtered['Longitude']
                                  )

Toronto_venues

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,Victoria Village,43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
2,Victoria Village,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Regent Park, Harbourfront",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Regent Park, Harbourfront",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
...,...,...,...,...,...,...,...
2119,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.739416,-79.588437,McDonald's,43.741757,-79.584230,Fast Food Restaurant
2120,"Mimico NW, The Queensway West, South of Bloor,...",43.706748,-79.594054,Economy Rent A Car,43.708471,-79.589943,Rental Car Location
2121,"Mimico NW, The Queensway West, South of Bloor,...",43.706748,-79.594054,Logistics Distribution,43.707554,-79.589252,Bar
2122,"Mimico NW, The Queensway West, South of Bloor,...",43.706748,-79.594054,Saand Rexdale,43.705072,-79.598725,Drugstore


now we have a dataframe with the needed data to work with

# Now, Let's cluster the neighborhoods

Run k-means to cluster the neighborhood into 5 clusters

In [38]:
from sklearn.cluster import KMeans

# set number of clusters
n_clusters = 5

cluster_df = Toronto_venues.drop(['Neighborhood', 'Venue', 'Venue Category'], axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(cluster_df)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

 The lab has used many deprecated features and i can't understand it to get this assignment done.
 I did my best for a week of trying please appreciate this in the peer graded assignment..